In [1]:
import numpy as np
%matplotlib inline
import matplotlib as plt

import worg

/home/scopatz/miniconda/lib/python3.5/importlib/_bootstrap.py:222: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)
/home/scopatz/miniconda/lib/python3.5/importlib/_bootstrap.py:222: QAWarning: pyne.material is not yet QA compliant.
  return f(*args, **kwds)
/home/scopatz/miniconda/lib/python3.5/importlib/_bootstrap.py:222: QAWarning: pyne.enrichment is not yet QA compliant.
  return f(*args, **kwds)


In [2]:
T = 50
seed = 424242
verbose = True

In [3]:
%%time
f = 90 * (1.01**np.arange(T))  # 1% growth
N = np.asarray(np.ceil(4*(1.01)**np.arange(T)), dtype=int)
state = worg.optimize(f, N, seed=seed, verbose=verbose, method_0='stochastic')

Simulation 3
------------
SimId 65b9d470-2800-4f73-8924-c6b785b6d24b
hyperparameters: [ 8.44155389  5.41756185]
Estimate method is 'stochastic'
Estimate winner is 'stochastic'
Estimate time:   0.0 min 6.491922855377197 sec
Simulation time: 0.0 min 7.2172510623931885 sec
D_s: 2.935123927440842
D: [2.9351239274408418, 14.802403115793279]

Simulation 4
------------
SimId bb0aa87a-2055-4698-8bc2-96d4eb824f21
hyperparameters: [ 7.56552878  4.3505483 ]
Estimate method is 'stochastic'
Estimate winner is 'stochastic'
Estimate time:   0.0 min 6.629546642303467 sec
Simulation time: 0.0 min 7.311304807662964 sec
D_s: 1.5142780541252059
D: [1.5142780541252059, 2.9351239274408418]

Simulation 5
------------
SimId 2b0a0340-007b-45f1-bc15-bd90a80ec59c
hyperparameters: [ 6.4493229   3.72067005]
Estimate method is 'stochastic'
Estimate winner is 'stochastic'
Estimate time:   0.0 min 6.415410280227661 sec
Simulation time: 0.0 min 8.617703437805176 sec
D_s: 1.0749170338271168
D: [1.0749170338271168, 1.51